In [131]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.preprocessing import Imputer as SimpleImputer
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import tree

In [132]:
def preprocessData(data):
    data = data.replace('Unknown', np.nan)
    data.dropna(inplace=True)
    
    le = preprocessing.LabelEncoder()
    for column_name in data.columns:
        if data[column_name].dtype == object:
            data[column_name] = le.fit_transform(data[column_name])
        else:
            pass
    
    return data

In [133]:
propertyData = pd.read_csv("property_prices.csv")
propertyData['lowBand'], propertyData['highBand'] = propertyData['price_bands'].str.split('-', 1).str
propertyData['lowBand'] = propertyData['lowBand'].str[:-1]
propertyData['highBand'] = propertyData['highBand'].str[:-1]
propertyData = preprocessData(propertyData)
propertyData_target = propertyData.lowBand
propertyData_predictors = propertyData.drop(['lowBand'], axis=1)
propertyData_numeric_predictors = propertyData_predictors.select_dtypes(exclude=['object'])

In [134]:
X_train, X_test, y_train, y_test = train_test_split(propertyData_numeric_predictors, 
                                                    propertyData_target,
                                                    train_size=0.7, 
                                                    test_size=0.3, 
                                                    random_state=0)
def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds)

In [135]:
cols_with_missing = [col for col in X_train.columns 
                                 if X_train[col].isnull().any()]
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_test  = X_test.drop(cols_with_missing, axis=1)
print("Mean Absolute Error from dropping columns with Missing Values:")
print(score_dataset(reduced_X_train, reduced_X_test, y_train, y_test))

Mean Absolute Error from dropping columns with Missing Values:
0.0


In [136]:
my_imputer = SimpleImputer()
imputed_X_train = my_imputer.fit_transform(X_train)
imputed_X_test = my_imputer.transform(X_test)
print("Mean Absolute Error from Imputation:")
print(score_dataset(imputed_X_train, imputed_X_test, y_train, y_test))

Mean Absolute Error from Imputation:
0.00018747656542932134


In [137]:
imputed_X_train_plus = X_train.copy()
imputed_X_test_plus = X_test.copy()
cols_with_missing = (col for col in X_train.columns 
                                 if X_train[col].isnull().any())
for col in cols_with_missing:
    imputed_X_train_plus[col + '_was_missing'] = imputed_X_train_plus[col].isnull()
    imputed_X_test_plus[col + '_was_missing'] = imputed_X_test_plus[col].isnull()
my_imputer = SimpleImputer()
imputed_X_train_plus = my_imputer.fit_transform(imputed_X_train_plus)
imputed_X_test_plus = my_imputer.transform(imputed_X_test_plus)
print("Mean Absolute Error from Imputation while Track What Was Imputed:")
print(score_dataset(imputed_X_train_plus, imputed_X_test_plus, y_train, y_test))

Mean Absolute Error from Imputation while Track What Was Imputed:
0.0


In [138]:
seed = 0
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cart = tree.DecisionTreeClassifier()
num_trees = 10
clf1 = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed ,max_features=3)
clf2 = RandomForestClassifier(n_estimators=num_trees, random_state=seed) 
clf3 = GaussianNB()
clf4 = AdaBoostClassifier(n_estimators=num_trees, random_state=seed) 
labels = ['Bagging', 'Random Forest', 'Naive Bayes', 'Ada Boost']
estimators = [clf1,clf2,clf3,clf4]
seed = 0
kfold = model_selection.KFold(n_splits=10, random_state=seed)
cart = tree.DecisionTreeClassifier()
num_trees = 10
vc = VotingClassifier(estimators=[('Bagging', clf1), ('Random Forest', clf2), ('Naive Bayes', clf3), ('Ada Boost', clf4)], voting='hard')
vc = vc.fit(X_test, y_test)
results = model_selection.cross_val_score(vc, X_test, y_test, cv=kfold) 
print("Hard Voting accuracy is", results.mean()*100, '%')
vc = VotingClassifier(estimators=[('Bagging', clf1), ('Random Forest', clf2), ('Naive Bayes', clf3), ('Ada Boost', clf4)], voting='soft')
vc = vc.fit(X_test, y_test)
results = model_selection.cross_val_score(vc, X_test, y_test, cv=kfold)
print("Soft Voting accuracy is", results.mean()*100, "%")

c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\sklearn\pr

Hard Voting accuracy is 99.96240601503759 %


c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\sklearn\pr

Soft Voting accuracy is 99.92495283151699 %


c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\owner\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
